In [1]:
 !nvidia-smi

Tue Sep 14 15:50:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 470.42.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:3B:00.0 Off |                  N/A |
|100%   91C    P2   115W / 250W |  10983MiB / 11019MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:5E:00.0 Off |                  N/A |
| 32%   

In [2]:
'''
Uncomment and install the packages 
'''
# %%capture
# !pip3 install git+https://github.com/huggingface/datasets.git
# !pip3 install git+https://github.com/huggingface/transformers.git
# !pip3 install torchaudio
# !pip3 install librosa
# !pip3 install jiwer
# !pip3 install pandas
# !pip3 install os

In [3]:
import os
import pandas as pd

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

In [5]:
path_to_dir_hindi_train = "/home/ankita/Data/Hindi_Data/Hindi/" # Replace accoriding to the file location
line_df_train_hin = pd.read_csv("/home/ankita/Data/Hindi_train_2.5.csv") # Replace with csv of required language

In [6]:
path_to_dir_hindi_test = '/home/ankita/Data/Hindi_Data/Hindi/' # Replace accoriding to the file location
line_df_test_hin = pd.read_csv("/home/ankita/Data/Hindi_dev.csv") # Replace with csv of required language

In [8]:
paths_train = []
texts_train = []
fils = os.listdir(path_to_dir_hindi_train)
for fil in fils:
    if '.wav' in fil:
        try:
            texts_train.append(line_df_train_hin[line_df_train_hin["audioFileName"]==fil]["text"].values[0])
            paths_train.append(os.path.join(path_to_dir_hindi_train,fil))
        except:
            continue

In [9]:
len(paths_train)

1500

In [10]:
paths_test = []
texts_test = []
fils = os.listdir(path_to_dir_hindi_test)
for fil in fils:
    if '.wav' in fil:
        try:
            texts_test.append(line_df_test_hin[line_df_test_hin["audioFileName"]==fil]["text"].values[0])
            paths_test.append(os.path.join(path_to_dir_hindi_test,fil))
        except:
            continue

In [11]:
from datasets import Dataset
train_dataset = Dataset.from_dict({"path":paths_train,"text":texts_train})
test_dataset = Dataset.from_dict({"path":paths_test,"text":texts_test})

In [12]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [13]:
show_random_elements(train_dataset.remove_columns(["path"]), num_examples=20)

,text
0,मंदिर लगभग साल पुराना है
1,इस दौरान यहूदी धर्म पर जरथुस्त धर्म का प्रभाव पड़ा
2,में के कोड का एक दूसरा संस्करण तैयार किया गया
3,उनका विवाह श्रीमती शिवा देवी के साथ हुआ था
4,प्राथमिक विधानों को संसदीय बहुमत द्वारा पारित किया जाता है
5,में उन्हेंमिला और वे सूबेदार बन गए
6,विज्ञान परिषद् के मुख्य कार्यक्षेत्र निम्नलिखित हैं
7,इनका जन्म एक कायस्थ परिवार में हुआ था
8,इससे रंग में हल्कापन आता है
9,स्थिति नियंत्रण से बाहर हो रही थी


In [14]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\…]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [15]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

In [16]:
show_random_elements(test_dataset.remove_columns(["path"]))

,text
0,ये परमाणु विखंडन के दौरान निकलते हैं
1,उन्हें यह बात राष्ट्र के खिलाफ लगने लगती है
2,वाहीक पंजाब के किसी पश्चिमीप्रदेश का नाम था
3,इससे कोईअप्रिय घटना घट सकती है
4,गोल्डमैनसैक्स अमेरिकी वित्तीय कंपनी है
5,मंत्री ने कहा `सुनाओ
6,इन्हींशब्दों के साथ मैं मा
7,कुल मिलाकर पर्सेंट टैक्स बनता है
8,स्व केन्द्रों द्वारा अर्जितराजस्व केवल १७
9,क्या तुम उससे नहीं मिलोगे रात का खाना खाकरचले जाना


In [17]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [18]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

In [19]:
vocab_list = list(set(vocab_train["vocab"][0])| set(vocab_test["vocab"][0]))

In [21]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

In [23]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [24]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

87

In [25]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [26]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [27]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value= 0.0 , do_normalize=True, return_attention_mask=True)

In [28]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [29]:
processor.save_pretrained("CLAP/wav2vec2-large-clsril-hi-2.5")

In [30]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    #batch["audio"] = speech_array
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

/home/ankita/venv/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [ ]:
train_dataset = train_dataset.map(speech_file_to_array_fn, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(speech_file_to_array_fn, remove_columns=test_dataset.column_names)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset))

ipd.Audio(data=np.asarray(train_dataset[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
print("Target text:", train_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", train_dataset[rand_int]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, batch_size=8, num_proc=1, batched=True)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, batch_size=8, num_proc=1, batched=True)

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import gc

gc.collect()

torch.cuda.empty_cache()

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximtrain_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, batch_size=8, num_proc=1, batched=False)
# test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, batch_size=8, num_proc=1, batched=False)um length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    return {"wer": wer,"cer": cer}

In [ ]:
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(
    "Harveenchadha/wav2vec2-pretrained-clsril-23-10k",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.65,
    layerdrop=0.05,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)+1,
)

In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir="CLAP/ckpts-clsril-hi-2.5/",
  logging_dir = "CLAP/runs-hi-clsril-hi-2.5/",
  group_by_length=True,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=90,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=0.00005,
  warmup_steps=int(0.1*1320), #10%
  save_total_limit=2
)

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("CLAP/final/wav2vec2-wav2vec2-large-clsril-hi-2.5")